In [43]:
import pandas as pd

# register/request from client
file = r"C:\Users\mkreidler\Desktop\ShellRegister.xlsx"
df = pd.read_excel(file)
df = df[["trc", "bv", "ship_status", "scope", "count", "Glaze Code", "Level"]]
df = df.loc[df["count"] == 1, :]
df["Glaze Code"] = df["Glaze Code"].str.replace("E", "A")
df["bvtc"] = df.bv + "-" + df["Glaze Code"]


# ctrac data for shell
ct = r"C:\Users\mkreidler\Desktop\shellall.xlsx"
ctrac = pd.read_excel(ct)
ctrac["Block ID"] = ctrac["Block ID"].str.replace("-E", "-A")

# append instance to each block id to have a unique join cell
def up(x):
    return x.upper()

df["instance"] = df.groupby("bvtc").cumcount() + 1
df["block_instance"] = df.bvtc + "[" + df.instance.map(str) + "]"
df.block_instance = df.block_instance.apply(up)

ctrac["instance"] = ctrac.groupby("Block ID").cumcount() + 1
ctrac["block_instance"] = ctrac["Block ID"] + "[" + ctrac.instance.map(str) + "]"
ctrac.block_instance = ctrac.block_instance.apply(up)

# merge both df on block_instance
merge = df.merge(ctrac, on="block_instance", how="outer")

# filter data
block_line = merge.loc[merge["Block Line Number"].isna(), :]
check = block_line.loc[block_line.ship_status == "check", :]
colors = list("AE")
ae = check.loc[check["Glaze Code"].str.contains("|".join(colors))]

len(check)


46

In [39]:
col = ["bvtc", "trc"] 

check = check.groupby("bvtc").count()
check.name = 0
check = check.reset_index()

check = check.loc[:,col]

check.trc.sum()

46

In [41]:
check



,bvtc,trc
0,A18A-A,1
1,A18A-C,1
2,A18A-D,1
3,A19-B,1
4,A19-F,1
5,A1A-A,5
6,A1A-B,5
7,A1A-C,4
8,A1A-D,2
9,A1A-F,2


In [44]:
check.to_excel(r"C:\Users\mkreidler\Desktop\shell_check_46.xlsx")